In [ ]:
import os
import json
import random
from glob import glob
from skimage import io, transform
from xml.etree.ElementTree import parse
from torch.utils.data import Dataset, DataLoader


class ImageNetDataloader(Dataset):
    def __init__(self, data_dir, category_dir = "imageNetCategory.json", random_rate = 0.0):
#         self.label_file = sorted(glob(label_dir +"/*.xml"))
        self.labels = []
        self.img_dirs = []
        for files in os.listdir(data_dir):
            for f in os.listdir(data_dir + "/"+files):
                self.labels.append(files)
                self.img_dirs.append(data_dir + "/" + files + "/" + f)
#         self.image_file = sorted(glob(img_dir +"/*.JPEG"))
        self.class_num = 0

        self.group = {}
        self.category_name = []        
        self.random_rate = random_rate # 바뀔 확률
        self.id2name = {}
        class_id = 0
        
        with open(category_dir, "r") as json_data:
            data = json.load(json_data)        

        
        oraganism_category = [
            data["children"][20]["children"][0], # plant
            data["children"][20]["children"][1], # fungus
            data["children"][20]["children"][2], # person
#             data["children"][20]["children"][3], # animal
            data["children"][20]["children"][3]["children"][0], # invertebrate 갑각류
#             data["children"][20]["children"][3]["children"][1], # domestic animal 가축
#             data["children"][20]["children"][3]["children"][2], # greyhound 경주용 개
            data["children"][20]["children"][3]["children"][3]["children"][0]["children"][0],#  mamal
            data["children"][20]["children"][3]["children"][3]["children"][0]["children"][1],#  bird
            data["children"][20]["children"][3]["children"][3]["children"][0]["children"][2],#  reptile
            data["children"][20]["children"][3]["children"][3]["children"][0]["children"][3],#  amphibian
            data["children"][20]["children"][3]["children"][3]["children"][0]["children"][4]#  aquatic vertebrate
        ]

        for d in (oraganism_category):
            leafnode = self.get_leafnode(d)
            same_class = [j[2] for j in leafnode]

            self.category_name.append(d['name'])

            for id,name, index in leafnode:
                if id in self.group.keys():
#                     print(name)
                    pass
                else:
                    self.group[id] = {"name": name, "class_id": index, "same_class" : same_class, 
                                      "category_name": d['name']}
                    self.id2name[index] = name
                    class_id+=1      

                
        
        for i in range(20):
            leafnode = self.get_leafnode(data["children"][i])
            same_class = [j[2] for j in leafnode]
            self.category_name.append(data["children"][i]['name'])

            for id,name,index in leafnode:
                if id in self.group.keys():
#                     print(name)
                    pass
                else:
                    self.group[id] = {"name": name, "class_id": index, "same_class" : same_class, 
                                      "category_name": data["children"][i]['name']}
                    self.id2name[index] = name
                    class_id+=1                
                
                
    def __len__(self):
        return len(self.lables)

    
    def __getitem__(self,idx):
        data = self.group[self.labels[idx]] #n****
        
        
        if random.random() <= self.random_rate:
            same_class = data["same_class"]
            label = random.randrange(0, len(same_class))
            label = same_class[label]
            while label == data["class_id"]:
                label = random.randrange(0, len(same_class))
                label = same_class[label]
#             label = same_class[0] + (((data["category_id"] - same_class[0]) + random.randrange(1,len(same_class)))% len(same_class))
#             print(data["category_name"])
#             print(same_class)
#             print(f"{data['class_id']} --> {label}")
#             print(f"{self.id2name[data['class_id']]} --> {self.id2name[label]}")
            img = io.imread(self.img_dirs[idx])
            return img, label
        else:
            img = io.imread(self.img_dirs[idx])
            return img, data["class_id"]
                                     
    def get_leafnode(self, data):
        id = []
        if "children" in data:
            for d in data["children"]:
                id += (self.get_leafnode(d))
            return id
        else:
            return [(data["id"], data["name"], int(data['index']))]


                                 
dataloader_train = ImageNetDataloader(data_dir = "train/", random_rate = 0.5)

dataloader_val = ImageNetDataloader(data_dir = "val/", random_rate = 0.5)